In [0]:
# Validando a SparkSession

spark

In [0]:
# Mostrando os pontos de montagem no cluster Databricks

display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/mnt/datalakeea44c854c651c494/gold,wasbs://gold@datalakeea44c854c651c494.blob.core.windows.net,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/datalakeea44c854c651c494/bronze,wasbs://bronze@datalakeea44c854c651c494.blob.core.windows.net,
/mnt/datalakeea44c854c651c494/landing-zone,wasbs://landing-zone@datalakeea44c854c651c494.blob.core.windows.net,
/Volume,DbfsReserved,
/mnt/datalakeea44c854c651c494/silver,wasbs://silver@datalakeea44c854c651c494.blob.core.windows.net,


In [0]:
# Desmontando os pontos de montagem se necessário

#dbutils.fs.unmount('/mnt/datalakeea44c854c651c494/lading-zone')
#dbutils.fs.unmount('/mnt/datalakeea44c854c651c494/bronze')
#dbutils.fs.unmount('/mnt/datalakeea44c854c651c494/silver')
#dbutils.fs.unmount('/mnt/datalakeea44c854c651c494/gold')

/mnt/datalakeea44c854c651c494/lading-zone has been unmounted.
/mnt/datalakeea44c854c651c494/bronze has been unmounted.
/mnt/datalakeea44c854c651c494/silver has been unmounted.
/mnt/datalakeea44c854c651c494/gold has been unmounted.


True

In [0]:
# Definindo uma função para montar um ADLS com um ponto de montagem com ADLS SAS

storageAccountName = "datalakeea44c854c651c494"
storageAccountAccessKey = ""
sasToken = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-20T02:32:31Z&st=2025-06-19T18:32:31Z&spr=https&sig=CGev8c8IkJuIFAGEeFukBHu%2B66q2MxFaeswMz0QNSVo%3D"

def mount_adls(blobContainerName):
    try:
      dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
        mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
        #extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
      )
      print(f"Montagem bem-sucedida: {blobContainerName}")
    except Exception as e:
      print(f"Falha na montagem de {blobContainerName}:", e)

In [0]:
# Montando todos os containers

mount_adls('lading-zone')
mount_adls('bronze')
mount_adls('silver')
mount_adls('gold')

Montagem bem-sucedida: lading-zone
Montagem bem-sucedida: bronze
Montagem bem-sucedida: silver
Montagem bem-sucedida: gold


In [0]:
# Mostrando os pontos de montagem no cluster Databricks

display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/mnt/datalakeea44c854c651c494/gold,wasbs://gold@datalakeea44c854c651c494.blob.core.windows.net,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/datalakeea44c854c651c494/bronze,wasbs://bronze@datalakeea44c854c651c494.blob.core.windows.net,
/mnt/datalakeea44c854c651c494/landing-zone,wasbs://landing-zone@datalakeea44c854c651c494.blob.core.windows.net,
/Volume,DbfsReserved,
/mnt/datalakeea44c854c651c494/silver,wasbs://silver@datalakeea44c854c651c494.blob.core.windows.net,


In [0]:
# Mostrando todos os arquivos da camada bronze

display(dbutils.fs.ls(f"/mnt/{storageAccountName}/bronze"))

path,name,size,modificationTime
dbfs:/mnt/datalakeea44c854c651c494/bronze/circuits/,circuits/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/constructor_results/,constructor_results/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/constructor_standings/,constructor_standings/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/constructors/,constructors/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/driver_standings/,driver_standings/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/drivers/,drivers/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/lap_times/,lap_times/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/pit_stops/,pit_stops/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/qualifying/,qualifying/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/races/,races/,0,0


In [0]:
# Gerando um dataframe dos delta lake no container bronze do Azure Data Lake Storage

df_circuits   = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/circuits")
df_constructor_results     = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/constructor_results")
df_constructor_standings   = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/constructor_standings")
df_constructors  = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/constructors")
df_driver_standings    = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/driver_standings")
df_drivers     = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/drivers")
df_lap_times     = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/lap_times")
df_pit_stops    = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/pit_stops")
df_qualifying = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/qualifying")
df_races    = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/races")
df_results  = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/results")
df_seasons  = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/seasons")
df_sprint_results  = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/sprint_results")
df_status  = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/status")

In [0]:
# Adicionando metadados de data e hora de processamento e nome do arquivo de origem

from pyspark.sql.functions import current_timestamp, lit

df_circuits = df_circuits.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("circuits"))
df_constructor_results   = df_constructor_results.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("constructor_results"))
df_constructor_standings = df_constructor_standings.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("constructor_standings"))
df_constructors  = df_constructors.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("constructors"))
df_driver_standings = df_driver_standings.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("driver_standings"))
df_drivers       = df_drivers.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("drivers"))
df_lap_times     = df_lap_times.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("lap_times"))
df_pit_stops     = df_pit_stops.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("pit_stops"))
df_qualifying    = df_qualifying.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("qualifying"))
df_races         = df_races.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("races"))
df_results       = df_results.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("results"))
df_seasons       = df_seasons.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("seasons"))
df_sprint_results = df_sprint_results.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("sprint_results"))
df_status        = df_status.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("status"))

In [0]:
# Listando os nomes das colunas dos dataframes

print(df_circuits.columns)
print(df_constructor_results.columns)
print(df_constructor_standings.columns)
print(df_constructors.columns)
print(df_driver_standings.columns)
print(df_drivers.columns)
print(df_lap_times.columns)
print(df_pit_stops.columns)
print(df_qualifying.columns)
print(df_races.columns)
print(df_results.columns)
print(df_seasons.columns)
print(df_sprint_results.columns)
print(df_status.columns)

['IDENTIFIER_CIRCUIT', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng', 'alt', 'url', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']
['constructorResultsId', 'raceId', 'constructorId', 'points', 'status', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']
['constructorStandingsId', 'raceId', 'constructorId', 'points', 'position', 'positionText', 'wins', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']
['constructorId', 'constructorRef', 'name', 'nationality', 'url', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']
['driverStandingsId', 'raceId', 'driverId', 'points', 'position', 'positionText', 'wins', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']
['driverId', 'driverRef', 'number', 'code', 'forename', 'surname', 'dob', 'nationality', 'url', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']
['raceId', 'driverId', 'lap', 'position', 'time', 'milliseconds', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']
['raceId', 'driverId',

In [0]:
# Mudando o nome das colunas para maiúscula e ajustanto os nomes das colunas de acordo com o dicionario de dados

#['circuitId', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng', 'alt', 'url', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver'] 
df_circuits = ( df_circuits 
               .withColumnRenamed("circuitId" , "IDENTIFIER_CIRCUIT")
               .withColumnRenamed("circuitRef" , "REFERENCE_CIRCUIT")
               .withColumnRenamed("name" , "NAME")
               .withColumnRenamed("location" , "LOCATION")
               .withColumnRenamed("country" , "COUNTRY" )
               .withColumnRenamed("lat" , "LATITUDE" )
               .withColumnRenamed("lng" , "LONGITUDE" )
               .withColumnRenamed("alt" , "ALTITUDE" )
               .withColumnRenamed("url" , "URL" )
               .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
               .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
               .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
               )
#['constructorResultsId', 'raceId', 'constructorId', 'points', 'status', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver'] 
df_constructor_results = ( df_constructor_results 
                          .withColumnRenamed("constructorResultsId" , "IDENTIFIER_CONSTRUCTOR_RESULTS")
                          .withColumnRenamed("raceId" , "IDENTIFIER_RACE")
                          .withColumnRenamed("constructorId" , "IDENTIFIER_CONSTRUCTOR")
                          .withColumnRenamed("points" , "POINTS")
                          .withColumnRenamed("status" , "STATUS")
                          .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
                          .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
                          .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
                          )
#['constructorStandingsId', 'raceId', 'constructorId', 'points', 'position', 'positionText', 'wins', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']
df_constructor_standings = ( df_constructor_standings 
                            .withColumnRenamed("constructorStandingsId" , "IDENTIFIER_CONSTRUCTOR_STANDINGS")
                            .withColumnRenamed("raceId" , "IDENTIFIER_RACE")
                            .withColumnRenamed("constructorId" , "IDENTIFIER_CONSTRUCTOR")
                            .withColumnRenamed("position" , "POSITION")
                            .withColumnRenamed("positionText" , "POSITION_TEXT")
                            .withColumnRenamed("wins" , "WINS")
                            .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
                            .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
                            .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
                            )
#['constructorId', 'constructorRef', 'name', 'nationality', 'url', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver'] 
df_constructors = ( df_constructors     
                   .withColumnRenamed("constructorId" , "IDENTIFIER_CONSTRUCTOR")
                   .withColumnRenamed("constructorRef" , "REFERENCE_CONSTRUCTOR")
                   .withColumnRenamed("name" , "NAME")
                   .withColumnRenamed("nationality" , "NATIONALITY")
                   .withColumnRenamed("url" , "URL")
                   .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
                   .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
                   .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
                   )
#['driverStandingsId', 'raceId', 'driverId', 'points', 'position', 'positionText', 'wins', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver'] 
df_driver_standings = ( df_driver_standings 
                       .withColumnRenamed("driverStandingsId" , "IDENTIFIER_DRIVER_STANDINGS")
                       .withColumnRenamed("raceId" , "IDENTIFIER_RACE")
                       .withColumnRenamed("driverId" , "IDENTIFIER_DRIVER")
                       .withColumnRenamed("points" , "POINTS")
                       .withColumnRenamed("position" , "POSITION")
                       .withColumnRenamed("positionText" , "POSITION_TEXT")
                       .withColumnRenamed("wins" , "WINS")
                       .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
                       .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
                       .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
                       )
#['driverId', 'driverRef', 'number', 'code', 'forename', 'surname', 'dob', 'nationality', 'url', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver'] 
df_drivers = ( df_drivers 
              .withColumnRenamed("driverId" , "IDENTIFIER_DRIVER")
              .withColumnRenamed("driverRef" , "REFERENCE_DRIVER")
              .withColumnRenamed("number" , "NUMBER")
              .withColumnRenamed("code" , "CODE")
              .withColumnRenamed("forename" , "FORENAME")
              .withColumnRenamed("surname" , "SURNAME")
              .withColumnRenamed("dob" , "DOB")
              .withColumnRenamed("nationality" , "NATIONALITY")
              .withColumnRenamed("url" , "URL")
              .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
              .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
              .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
              )
#['raceId', 'driverId', 'lap', 'position', 'time', 'milliseconds', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']            
df_lap_times = ( df_lap_times 
                .withColumnRenamed("RACEID" , "IDENTIFIER_RACE")
                .withColumnRenamed("driverId" , "IDENTIFIER_DRIVER")
                .withColumnRenamed("lap" , "LAP")
                .withColumnRenamed("position" , "POSITION")
                .withColumnRenamed("time" , "TIME")
                .withColumnRenamed("milliseconds" , "MILLISECONDS")
                .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
                .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
                .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
                )
#['raceId', 'driverId', 'stop', 'lap', 'time', 'duration', 'milliseconds', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']             
df_pit_stops = ( df_pit_stops
                .withColumnRenamed("raceId" , "IDENTIFIER_RACE")
                .withColumnRenamed("driverId" , "IDENTIFIER_DRIVER")
                .withColumnRenamed("stop" , "STOP")
                .withColumnRenamed("lap" , "LAP")
                .withColumnRenamed("time" , "TIME")
                .withColumnRenamed("duration" , "DURATION")
                .withColumnRenamed("milliseconds" , "MILLISECONDS")
                .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
                .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
                .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
                )
#['qualifyId', 'raceId', 'driverId', 'constructorId', 'number', 'position', 'q1', 'q2', 'q3', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver'] 
df_qualifying = ( df_qualifying 
                 .withColumnRenamed("qualifyId" , "IDENTIFIER_QUALIFYING")
                 .withColumnRenamed("raceId" , "IDENTIFIER_RACE")
                 .withColumnRenamed("driverId" , "IDENTIFIER_DRIVER")
                 .withColumnRenamed("constructorId" , "IDENTIFIER_CONSTRUCTOR")
                 .withColumnRenamed("number" , "NUMBER")
                 .withColumnRenamed("position" , "POSITION")
                 .withColumnRenamed("q1" , "Q1")
                 .withColumnRenamed("q2" , "Q2")
                 .withColumnRenamed("q3" , "Q3")
                 .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
                 .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
                 .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
                 )
#['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver'] 
df_races = ( df_races 
            .withColumnRenamed("RACEID" , "IDENTIFIER_RACE")
            .withColumnRenamed("year" , "YEAR")
            .withColumnRenamed("round" , "ROUND")
            .withColumnRenamed("circuitId" , "IDENTIFIER_CIRCUIT")
            .withColumnRenamed("name" , "NAME")
            .withColumnRenamed("date" , "DATE")
            .withColumnRenamed("time" , "TIME")
            .withColumnRenamed("url" , "URL")
            .withColumnRenamed("fp1_date" , "FP1_DATE")
            .withColumnRenamed("fp1_time" , "FP1_TIME")
            .withColumnRenamed("fp2_date" , "FP2_DATE")
            .withColumnRenamed("fp2_time" , "FP2_TIME")
            .withColumnRenamed("fp3_date" , "FP3_DATE")
            .withColumnRenamed("fp3_time" , "FP3_TIME")
            .withColumnRenamed("quali_date" , "QUALI_DATE")
            .withColumnRenamed("quali_time" , "QUALI_TIME")
            .withColumnRenamed("sprint_date" , "SPRINT_DATE")
            .withColumnRenamed("sprint_time" , "SPRINT_TIME")
            .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
            .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
            .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
            )
#['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid', 'position', 'positionText', 'positionOrder', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver'] 
df_results = ( df_results 
              .withColumnRenamed("resultId" , "IDENTIFIER_RESULT")
              .withColumnRenamed("raceId" , "IDENTIFIER_RACE")
              .withColumnRenamed("driverId" , "IDENTIFIER_DRIVER")
              .withColumnRenamed("constructorId" , "IDENTIFIER_CONSTRUCTOR")
              .withColumnRenamed("number" , "NUMBER")
              .withColumnRenamed("grid" , "GRID")
              .withColumnRenamed("position" , "POSITION")
              .withColumnRenamed("positionText" , "POSITION_TEXT")
              .withColumnRenamed("positionOrder" , "POSITION_ORDER")
              .withColumnRenamed("points" , "POINTS")
              .withColumnRenamed("laps" , "LAPS")
              .withColumnRenamed("time" , "TIME")
              .withColumnRenamed("milliseconds" , "MILLISECONDS")
              .withColumnRenamed("fastestLap" , "FASTESTLAP")
              .withColumnRenamed("rank" , "RANK")
              .withColumnRenamed("fastestLapTime" , "FASTESTLAPTIME")
              .withColumnRenamed("fastestLapSpeed" , "FASTESTLAPSPEED")
              .withColumnRenamed("statusId" , "IDENTIFIER_STATUS")
              .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
              .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
              .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
              )
#['year', 'url', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']               
df_seasons = (df_seasons 
              .withColumnRenamed("year" , "YEAR")
              .withColumnRenamed("url" , "URL")
              .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
              .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
              .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER")
              )
#['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid', 'position', 'positionText', 'positionOrder', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 'fastestLapTime', 'statusId', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver'] 
df_sprint_results = ( df_sprint_results
                     .withColumnRenamed("resultId" , "IDENTIFIER_RESULT")
                     .withColumnRenamed("raceId" , "IDENTIFIER_RACE")
                     .withColumnRenamed("driverId" , "IDENTIFIER_DRIVER")
                     .withColumnRenamed("constructorId" , "IDENTIFIER_CONSTRUCTOR")
                     .withColumnRenamed("number" , "NUMBER")
                     .withColumnRenamed("grid" , "GRID")
                     .withColumnRenamed("position" , "POSITION")
                     .withColumnRenamed("positionText" , "POSITION_TEXT")
                     .withColumnRenamed("positionOrder" , "POSITION_ORDER")
                     .withColumnRenamed("points" , "POINTS")
                     .withColumnRenamed("laps" , "LAPS")
                     .withColumnRenamed("time" , "TIME")
                     .withColumnRenamed("milliseconds" , "MILLISECONDS")
                     .withColumnRenamed("fastestLap" , "FASTESTLAP")
                     .withColumnRenamed("fastestLapTime" , "FASTESTLAPTIME")
                     .withColumnRenamed("statusId" , "IDENTIFIER_STATUS")
                     .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
                     .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
                     .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER")
                     )
#['statusId', 'status', 'data_hora_bronze', 'nome_arquivo', 'data_hora_silver']
df_status = ( df_status
            .withColumnRenamed("statusId" , "IDENTIFIER_STATUS")
            .withColumnRenamed("status" , "STATUS")
            .withColumnRenamed("data_hora_bronze", "DATA_HORA_BRONZE") 
            .withColumnRenamed("nome_arquivo" , "NOME_ARQUIVO") 
            .withColumnRenamed("data_hora_silver" , "DATA_HORA_SILVER") 
            )

In [0]:
# Consultando os dados alterados

print(df_circuits.columns)
print(df_constructor_results.columns)
print(df_constructor_standings.columns)
print(df_constructors.columns)
print(df_driver_standings.columns)
print(df_drivers.columns)
print(df_lap_times.columns)
print(df_pit_stops.columns)
print(df_qualifying.columns)
print(df_races.columns)
print(df_results.columns)
print(df_seasons.columns)
print(df_sprint_results.columns)
print(df_status.columns)

['IDENTIFIER_CIRCUIT', 'REFERENCE_CIRCUIT', 'NAME', 'LOCATION', 'COUNTRY', 'LATITUDE', 'LONGITUDE', 'ALTITUDE', 'URL', 'DATA_HORA_BRONZE', 'NOME_ARQUIVO', 'DATA_HORA_SILVER']
['IDENTIFIER_CONSTRUCTOR_RESULTS', 'IDENTIFIER_RACE', 'IDENTIFIER_CONSTRUCTOR', 'POINTS', 'STATUS', 'DATA_HORA_BRONZE', 'NOME_ARQUIVO', 'DATA_HORA_SILVER']
['IDENTIFIER_CONSTRUCTOR_STANDINGS', 'IDENTIFIER_RACE', 'IDENTIFIER_CONSTRUCTOR', 'points', 'POSITION', 'POSITION_TEXT', 'WINS', 'DATA_HORA_BRONZE', 'NOME_ARQUIVO', 'DATA_HORA_SILVER']
['IDENTIFIER_CONSTRUCTOR', 'REFERENCE_CONSTRUCTOR', 'NAME', 'NATIONALITY', 'URL', 'DATA_HORA_BRONZE', 'NOME_ARQUIVO', 'DATA_HORA_SILVER']
['IDENTIFIER_DRIVER_STANDINGS', 'IDENTIFIER_RACE', 'IDENTIFIER_DRIVER', 'POINTS', 'POSITION', 'POSITION_TEXT', 'WINS', 'DATA_HORA_BRONZE', 'NOME_ARQUIVO', 'DATA_HORA_SILVER']
['IDENTIFIER_DRIVER', 'REFERENCE_DRIVER', 'NUMBER', 'CODE', 'FORENAME', 'SURNAME', 'DOB', 'NATIONALITY', 'URL', 'DATA_HORA_BRONZE', 'NOME_ARQUIVO', 'DATA_HORA_SILVER']
['I

In [0]:
# Visualizando um dataframe atualizado para silver

df_status.display()

IDENTIFIER_STATUS,STATUS,DATA_HORA_BRONZE,NOME_ARQUIVO,DATA_HORA_SILVER
1,Finished,2025-06-19T21:54:27.228Z,status,2025-06-19T23:40:31.623Z
2,Disqualified,2025-06-19T21:54:27.228Z,status,2025-06-19T23:40:31.623Z
3,Accident,2025-06-19T21:54:27.228Z,status,2025-06-19T23:40:31.623Z
4,Collision,2025-06-19T21:54:27.228Z,status,2025-06-19T23:40:31.623Z
5,Engine,2025-06-19T21:54:27.228Z,status,2025-06-19T23:40:31.623Z
6,Gearbox,2025-06-19T21:54:27.228Z,status,2025-06-19T23:40:31.623Z
7,Transmission,2025-06-19T21:54:27.228Z,status,2025-06-19T23:40:31.623Z
8,Clutch,2025-06-19T21:54:27.228Z,status,2025-06-19T23:40:31.623Z
9,Hydraulics,2025-06-19T21:54:27.228Z,status,2025-06-19T23:40:31.623Z
10,Electrical,2025-06-19T21:54:27.228Z,status,2025-06-19T23:40:31.623Z


In [0]:
# Salvando os dataframes em delta lake (formato de arquivo) no Data Lake (repositorio cloud)

df_circuits.write.format('delta').save(f"/mnt/{storageAccountName}/silver/circuits")
df_constructor_results.write.format('delta').save(f"/mnt/{storageAccountName}/silver/constructor_results")
df_constructor_standings.write.format('delta').save(f"/mnt/{storageAccountName}/silver/constructor_standings")
df_constructors.write.format('delta').save(f"/mnt/{storageAccountName}/silver/constructors")
df_driver_standings.write.format('delta').save(f"/mnt/{storageAccountName}/silver/driver_standings")
df_drivers.write.format('delta').save(f"/mnt/{storageAccountName}/silver/drivers")
df_lap_times.write.format('delta').save(f"/mnt/{storageAccountName}/silver/lap_times")
df_pit_stops.write.format('delta').save(f"/mnt/{storageAccountName}/silver/pit_stops")
df_qualifying.write.format('delta').save(f"/mnt/{storageAccountName}/silver/qualifying")
df_races.write.format('delta').save(f"/mnt/{storageAccountName}/silver/races")
df_results.write.format('delta').save(f"/mnt/{storageAccountName}/silver/results")
df_seasons.write.format('delta').save(f"/mnt/{storageAccountName}/silver/seasons")
df_sprint_results.write.format('delta').save(f"/mnt/{storageAccountName}/silver/sprint_results")
df_status.write.format('delta').save(f"/mnt/{storageAccountName}/silver/status")

In [0]:
# Verificando os dados gravados em delta na camada SILVER

display(dbutils.fs.ls(f"/mnt/{storageAccountName}/silver/"))

path,name,size,modificationTime
dbfs:/mnt/datalakeea44c854c651c494/silver/circuits/,circuits/,0,0
dbfs:/mnt/datalakeea44c854c651c494/silver/constructor_results/,constructor_results/,0,0
dbfs:/mnt/datalakeea44c854c651c494/silver/constructor_standings/,constructor_standings/,0,0
dbfs:/mnt/datalakeea44c854c651c494/silver/constructors/,constructors/,0,0
dbfs:/mnt/datalakeea44c854c651c494/silver/driver_standings/,driver_standings/,0,0
dbfs:/mnt/datalakeea44c854c651c494/silver/drivers/,drivers/,0,0
dbfs:/mnt/datalakeea44c854c651c494/silver/lap_times/,lap_times/,0,0
dbfs:/mnt/datalakeea44c854c651c494/silver/pit_stops/,pit_stops/,0,0
dbfs:/mnt/datalakeea44c854c651c494/silver/qualifying/,qualifying/,0,0
dbfs:/mnt/datalakeea44c854c651c494/silver/races/,races/,0,0


In [0]:
# Consultando um exemplo de um delta lake para validar a existência dos dados e das colunas do metadados na camada SILVER

spark.read.format('delta').load(f'/mnt/{storageAccountName}/silver/drivers').limit(10).display()

IDENTIFIER_DRIVER,REFERENCE_DRIVER,NUMBER,CODE,FORENAME,SURNAME,DOB,NATIONALITY,URL,DATA_HORA_BRONZE,NOME_ARQUIVO,DATA_HORA_SILVER
1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,2025-06-19T21:54:03.058Z,drivers,2025-06-19T23:41:52.278Z
2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,2025-06-19T21:54:03.058Z,drivers,2025-06-19T23:41:52.278Z
3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg,2025-06-19T21:54:03.058Z,drivers,2025-06-19T23:41:52.278Z
4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,2025-06-19T21:54:03.058Z,drivers,2025-06-19T23:41:52.278Z
5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen,2025-06-19T21:54:03.058Z,drivers,2025-06-19T23:41:52.278Z
6,nakajima,\N,NAK,Kazuki,Nakajima,1985-01-11,Japanese,http://en.wikipedia.org/wiki/Kazuki_Nakajima,2025-06-19T21:54:03.058Z,drivers,2025-06-19T23:41:52.278Z
7,bourdais,\N,BOU,Sébastien,Bourdais,1979-02-28,French,http://en.wikipedia.org/wiki/S%C3%A9bastien_Bourdais,2025-06-19T21:54:03.058Z,drivers,2025-06-19T23:41:52.278Z
8,raikkonen,7,RAI,Kimi,Räikkönen,1979-10-17,Finnish,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen,2025-06-19T21:54:03.058Z,drivers,2025-06-19T23:41:52.278Z
9,kubica,88,KUB,Robert,Kubica,1984-12-07,Polish,http://en.wikipedia.org/wiki/Robert_Kubica,2025-06-19T21:54:03.058Z,drivers,2025-06-19T23:41:52.278Z
10,glock,\N,GLO,Timo,Glock,1982-03-18,German,http://en.wikipedia.org/wiki/Timo_Glock,2025-06-19T21:54:03.058Z,drivers,2025-06-19T23:41:52.278Z
